In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


Dense katmanı(38,Softmax) olarak belirttiğimiz zaman yaptığı şey 38 farklı sınıf içerisinden her sınıf için olasılık verip toplamının 1 olmasını sağlıyor

ML de predict_proba mantığına benzer ama onun multiclass classification versiyonu gibi dusunulebilir

In [ ]:
print(os.listdir("/kaggle/input"))

Kaggle içinde eklediğimiz veriseti için kontrol yapıyoruz

In [ ]:
base_path = "/kaggle/input/new-plant-diseases-dataset"
print("İçerik:")
print(os.listdir(base_path))

Bu dataset içinde bulunan dosyalara bir bakış attık.

In [ ]:
print("Klasörler:", os.listdir(base_path))

print("\nİçerik (New Plant Diseases Dataset(Augmented)) için:")
print(os.listdir(os.path.join(base_path, "New Plant Diseases Dataset(Augmented)"))[:5])
#Burada base path adlı klasore git ve onun altındaki 2.parametre olarak girdiğimiz klasörunu aç demek os.path.join bu şekil parçaları birleştiriyor

print("\nİçerik (new plant diseases dataset(augmented) için ise):")
print(os.listdir(os.path.join(base_path, "new plant diseases dataset(augmented)"))[:5])

#os.listdir ise:Verilen klasörün içindeki dosya ve klasörlerin isimlerini liste olarak döndürür. :5 ise basit slicing işlem

Burada bir de bu en son dosya yapısının altında train ve valid saklanmakta onları bu şekilde tespit etmiş olduk ve dosya içinde dosya olduğunu anladık işlemlerimizi buna göre gerçekleştireceğiz.

In [ ]:
dataset_path = "/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)"

train_dir = dataset_path + "/train"
val_dir   = dataset_path + "/valid"
test_dir  = base_path + "/test"

burada uygun formatı dosyaya taşıdık

In [ ]:
#DİKKAT :   BURADA BULUNAN CEVAPLAR dir... ASLINDA STRİNG OLDUĞU İÇİN ONUN STRİNG UZUNLUGUNU VERİYOR ARAMAK İSTEDİĞİMİZ CEVABI DEĞİL
print(len(train_dir))
print(len(val_dir))
print(len(test_dir))

Bu doğru olan bir çıktı araması değildir ve bu şekilde sadece verdiğimiz veriyolunun uzunlugunu donduruyor.

In [ ]:
#İstediğim cevap için:
print("Train sınıf sayısı:", len(os.listdir(train_dir)))
print("Valid sınıf sayısı:", len(os.listdir(val_dir)))
print("Test sınıf sayısı:", len(os.listdir(test_dir)))

In [ ]:
test_count = sum([len(files) for r, d, files in os.walk(test_dir)])
print("Test örnek sayısı:", test_count)


Burada herbiri için sınıf sayısını bastırdık ve kontrol sağladık.

In [ ]:
from tensorflow import keras

IMG_SIZE   = (224, 224)  # resim boyutu
BATCH_SIZE = 32
SEED       = 42

train_ds = keras.utils.image_dataset_from_directory(train_dir,image_size=IMG_SIZE,batch_size=BATCH_SIZE,
    label_mode="categorical",    #burada one hot vektor kullanılıyor cunku 38 sınıf var ve gerekli oluyor o yuzden de categorical seçildi.
    seed=SEED    #shuffle işlemi sabitliği için, random_state gibi
)


val_ds = keras.utils.image_dataset_from_directory(val_dir,image_size=IMG_SIZE,batch_size=BATCH_SIZE,label_mode="categorical",seed=SEED)


test_ds = keras.utils.image_dataset_from_directory(test_dir,image_size=IMG_SIZE,batch_size=BATCH_SIZE,label_mode="categorical",shuffle=False) 
#tahminlerin ve resimlerin birbirine karısmaması için shuffle kapalıdır.(test aşaması için)

Burada keras.utils.image_dataset_from_directory() yapısı ile klasor yapısında bulunan gorselleri okuyup etiketleri çıkartıyor.
ve train seti için image_dataset_from_directory() kısmında shuffle default olarak true değerindedir bu yuzden ekstra olarak belirtmeme gerek yok.
shuffle kullanımı arka arka gelen resimler sonucunda modelin ezber yapmasının önune geçmek /n

keras.utils.image_dataset_from_directory() yapısından sonra elde ettiğimiz şey aslında artık kerasta dataset objemiz oluyor 
Yani: TensorFlow’un kendi “dataset sınıfı.”

Özellikleri:
İçinde batch’ler halinde resimler + etiketler var.
Lazy evaluation (yani her seferinde batch hazırlar, RAM’i doldurmaz).
GPU/TPU için optimize edilmiş.

In [ ]:
num_classes = len(train_ds.class_names) 
print(num_classes)

train_ds dosyasını tekrardan kontrol ettik.(gizli dosya olma ihtimaline karşı bir kontrol için yaptık bunu)

In [ ]:
for images, labels in train_ds.take(1):      # take 1 ile  sadece 1 batchi al dedik
    print("Resim batch boyutu:", images.shape)
    print("Etiket batch boyutu:", labels.shape)
    print("İlk etiket örneği:", labels[0])

Dataset aslında bir tuple (images, labels) döndürür./n* 
images → (32, 224, 224, 3) boyutunda(32 tane 224x224 RGB resim)./n
labels → (32, 38) boyutunda (32 resmin one-hot encoded etiketleri) yani batch size için 32 resim vermiştik ve bunlar 38 sınıfa dağılcak onu gösteriyor ve sadece ait olduğu o sınıf için 1 değerini alıcak./n

label değerlerinin o şekilde olmasının sebebi label mode categorical ayarladıgımız için gerçelkleşti.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Sınıfları listelemek istersek:
diseases = os.listdir(os.path.join(dataset_path, "train"))

print("Sınıf sayısı:", len(diseases))
print("Hastalıklar:", diseases)


Burada bastırdığımız sınıf isimleri hakkında bilgilendirme yaptım:

> Blight:	Yaprak, dal veya meyvede ani kararma,
> kurumaRust:	Yaprakta pas renginde lekeler
> Scab:	Meyve veya yaprakta kabuklu, sert lezyon
> Mildew:	Beyazımsı mantar tabakası (kuf)
> Leaf spot:Yaprakta yuvarlak lekeler
> Curl Virus:	Yaprakların kıvrılması, şekil bozulması
> Mosaic Virus:	Yaprakta renkli desen bozulması
> Canker:	Dal veya gövdede çökük, yaralı bölge
> Greening (Haunglongbing): Meyve kucuk,şekilsiz,acı olur*
 	

In [ ]:
plants = []
NumberOfDiseases = 0
for plant in diseases:
    if plant.split('___')[0] not in plants:
        plants.append(plant.split('___')[0])
    if plant.split('___')[1] != 'healthy':
        NumberOfDiseases += 1

bu şekilde kullanılan bitkilerin isimleri ile toplam hastalık sayısını elde etmiş olduk

In [ ]:
print("Bitkiler:\n",plants)
print()
print("Bitki sayısı:", len(plants))

In [ ]:
print("Hastalık sayısı :",NumberOfDiseases)

In [ ]:
nums = {}
for disease in diseases:     #bu diseases aynı zamanda sutun adı anlamına da gelmekte
    nums[disease] = len(os.listdir(train_dir + '/' + disease))
    
img_per_class = pd.DataFrame(nums.values(), index=nums.keys(), columns=["number of images"])
img_per_class

In [ ]:
bu yazdığım kod ile hastalığa sahip olan sutunların içerdiği veri sayısını görmus oluyorum

In [ ]:
import matplotlib.pyplot as plt

def show_image(image, label, class_names):
    plt.imshow(image.numpy().astype("uint8"))   # resmi numpy'a çevir 255 piksel değerlikli  uint8 ,   imshow → matplotlib’e “şu veriyi(numpy arrayeçevrilmiş ve 255 değerlikli sayıları) resim olarak göster” diyor.
    plt.title("Label: " + class_names[label])   # etiket ismini yaz
    plt.axis("off") # bu da köşe kenarları siliyor
    plt.show()


HATIRLATMA :  biz train_dir ,val_dir gibi yapıları train_ds ,val_ds gibi yapılara keras.utils.dataset_from_directory() yapısı ile çevirdikten sonra zaten burada image ve label olarak tutuluyorlar 
-. image :  0-255 piksel değerlerine sahip bir matris iken; 
-. label : onun etiketi  label='cathegorical ' olarak seçildiyse labelın tutulma şekli one hot vektor [0,0,0,0,1...] şeklinde dir eğer label =int  ise bu sefer sınıf no=4 olarak tutulur mesela

In [ ]:
class_names = train_ds.class_names  # klasör isimlerini al
for images, labels in train_ds.take(1):  # bir batch al batch_size=32 olduğu için 32 adet aldık
    show_image(images[0], labels[0].numpy().argmax(), class_names)    #arg max ile sınıfını tespit etmiş oluyoruz buyuklugunu bularak


Bir tane batch aldık 32 olarak ayarlamıstık buradan 32 resimi aldık teker teker dönmeye başladı resimler ve onların içerisinden ilk resmi bastırdık.

In [ ]:
import numpy as np

def show_batch(dataset, class_names):
    plt.figure(figsize=(12, 12))
    for images, labels in dataset.take(1):  # ilk batch
        for i in range(9):  # ilk 9 resmi çizelim
            ax = plt.subplot(3, 3, i + 1)    #burada da ax olarak değişken olarak tutuyoruz o veriyi ama amacı ax.set_title falan  ayarlamak için o şekilde yapıyoruz.
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title(class_names[labels[i].numpy().argmax()])
            plt.axis("off")

ax = plt.subplot(3, 3, i + 1) nedir?
->plt.subplot(nrows, ncols, index) = Grafiği küçük kutucuklara böler.
nrows=3 → 3 satır
ncols=3 → 3 sütun
index=i+1 → şu an hangi kutucuğa çizileceğini gösterir.   1 mesela sol en ust anlamında 3x3  luk ızgara çizdirdik.

In [ ]:
show_batch(train_ds, train_ds.class_names)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# Data Augmentation yapalım ve verimizin gerçek hayata karşı daha dayanıklı hale getirelim.
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),   # resmi yatay çevir
    layers.RandomRotation(0.1),        # ±%10 döndür
    layers.RandomZoom(0.1),            # biraz zoom
    layers.RandomBrightness(factor=0.1)  # parlaklık değiştir
])


İmageDataGenarator(cpu taban) yerine keras data augmentation(GPU tabanı)uzerinden bunu kullandım ve data augmentation işlemini bu şekilde gerçekleştirdim parametreleri de sık kullanılan parametreler olarak ayarladım
1. Sequantial burada bir pipeline gibi davranır her katman göruntuyu bir adım donusturur. 

In [ ]:
# CNN Modeli
model = keras.Sequential([
    layers.Input(shape=(224, 224, 3)),  # giriş: 224x224 RGB resim bu değer standart değerdir ve çok sık kullanılır
    
    data_augmentation,                  # augmentation bloğunu burada ekliyoruz ve en başta resim uzerinde(flip-rotate-zoom-bright) işlemlerini ekliyor.
    layers.Rescaling(1./255),  # normalize edelim DAHA SONRASINDA EKLEDİM KODA DAHA CNN 0-1 ARASINDA DAHA İYİ ÇALIŞIYORMUŞ O yuzden ekledim.

    # 1. convolution bloğu
    layers.Conv2D(32, (3,3), activation="relu", padding="same"),   #32 tane feature detector (kernel)   , act. fonk. =relu olması nedeniyle nonlinearlik başlıyor
    layers.MaxPooling2D(),

    # 2. convolution bloğu
    layers.Conv2D(64, (3,3), activation="relu", padding="same"),
    layers.MaxPooling2D(),
    layers.Dropout(0.20),      #eğer çok yuksek verirsek ogrenme yavaslar ama cok dusuk verirsekte overfitting riski bulunur -> Noronların %25 ini kapat anlamında 

    # 3. convolution bloğu               (Datasetteki resim sayısı ve sınıf sayısına göre conv katman sayısını belirliyoruz)
    layers.Conv2D(128, (3,3), activation="relu", padding="same"),
    layers.MaxPooling2D(),
    layers.Dropout(0.20),


    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.4),   # Fully connected kısmında overfitting azaltmak için daha yuksek bir dropout kullandk

    # Çıkış katmanı: 38 sınıf (softmax → olasılık dağılımı)
    layers.Dense(38, activation="softmax")
])


model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),                                                   #'Adam':Modelin öğrenme hızını ayarlıyor(pratik ve hızlı)(değiştim)
    loss="categorical_crossentropy",                                      #bunu kullanmamızın nedeni ise çok fazla sınıf sayısına sahip olmamız eğer one hot kullansak loss kullanacakmışız
    metrics=["accuracy"]
)

# Model özeti
model.summary()  

Model summary ile ;
1. Modelde olan her katman listelendi.
2. Output Shape kısmında none=>Batch Size(kaç resim aynı anda verildi) verilmediği için öyle gözukuyor , ve 32 filtre(kanal )anlamına gelmekte
3. Poolingin öğrenilebilecek parametresi olmadığı için param değeri 0 olarak gözukuyor sadece amacı kucultme yapmak
4. dense parametre hesabı = 100352*128+128   (giriş boyutu x çıkış boyutu + bias)
5. dense(son) parametre sayısı: 128*38+38
6. en alttaki toplam parametre modelin karmaşıklığını gösterir.  Parametre hesabı ise şu şekilde yapılır: ((kernel boyutları carpımı(3x3 vb.) * input_channel(RGB)=3) + 1) * filtre sayısı

-> 6 DA + 1 KISMI BIAS için geldi çunku her filtre için 1 tane bias ekliyoruz.       conv2d için hesap:     (((3x3) x 3 )+1)*32=896 olur.



NOT: NORMALDE RESCALİNG İŞLEMİNİ AUGMENTATİONDAN ÖNCE GERÇEKLEŞTİRMİŞTİM AMA 'RandomBrightness gibi augment işlemleri 0–1 aralığında değil, 0–255 piksel aralığında bekliyormuş'. o yuzden augmentationdan 
sonraya koydum.

! Dikkat edilmesi gereken noktalardan biri ise padding='same' olarak seçildiği zaman conv2d katmanından sonra output size uzerinden bir azalma durumu yaşanmaz

In [ ]:
# MODELİ EĞİTME KISMINA GEÇELİM:
history = model.fit(
    train_ds,             
    validation_data=val_ds, 
    epochs=10             
    # başlangıç için 10 epoch denemesi yaptım daha sonrasında 12 ye cıkardım cunku asırı 10 epochta hala accurasy gittikçe artıyordu ama accurasy değeri ciddi azaldığını gordum ve 10 epocha geri döndum.
)


İlk denemememiz için ortalama bir sonuçmuş epoch sayısını arttırıp daha iyi bir sonuç çıkartılabilir. Şimdi görselleştrime asamasına geçelim.

1. train_ds ile öğrendi, val_ds ile her epoch sonunda kendini doğruladı.
2. keras bu eğitim sonunda bir history nesnesi döner , bu bir class history örneğiymiş.
3. history nesnesinin içinde .history adlı attribute vardır ve eğitimde öğrenilen bilgileri tutan bir sözluk yapısı.Bu nesne şunları içerir:
.params → eğitim parametreleri (batch size, epoch sayısı vs.)
.epoch → epoch numaraları
.model → eğitilen modelin kendisi
.history → metriklerin tutulduğu sözlük
Yani history bir “kapsayıcı” nesnedir.

In [ ]:
import json
with open("history.json", "w") as f:
    json.dump(history.history, f)

In [ ]:
import json
with open("history.json",'r') as f:
    history_data=json.load(f)
print(history_data)

In [ ]:
df = pd.DataFrame(history.history)
df.index += 1  # Epoch numaraları 1’den başlasın
df.index.name = "Epoch"
print(df)


In [ ]:
import matplotlib.pyplot as plt

history=history_data
acc      = history["accuracy"]
val_acc  = history["val_accuracy"]
loss     = history["loss"]
val_loss = history["val_loss"]
epochs   = range(1, len(acc)+1)

plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(epochs, acc, label="train")
plt.plot(epochs, val_acc, label="val")
plt.title("Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs, loss, label="train")
plt.plot(epochs, val_loss, label="val")
plt.title("Loss")
plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend()

plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# --- 1) Gerçek ve tahmin etiketlerini topla ---
y_true = np.concatenate([y.numpy() for x, y in test_ds], axis=0)
y_pred = np.concatenate([model.predict(x) for x, y in test_ds], axis=0)

# One-hot → sınıf indeksine çevir
y_true = y_true.argmax(axis=1)
y_pred = y_pred.argmax(axis=1)

# --- 2) Sadece gerçekten testte bulunan sınıfları al ---
labels_present = np.unique(y_true)  # testte gerçekten olan sınıflar
names_present = [class_names[i] for i in labels_present]  # o sınıfların isimleri

# --- 3) Classification Report ---
print(classification_report(
    y_true, y_pred,
    labels=labels_present,
    target_names=names_present,
    zero_division=0   # 0 bölme hatalarını engeller
))

# --- 4) Confusion Matrix ---
cm = confusion_matrix(y_true, y_pred, labels=labels_present)
plt.figure(figsize=(12,10))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=names_present, yticklabels=names_present)
plt.title("Confusion Matrix (Test Set)")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()


TEST VERİSETİNDE SADECE 33 ORNEK VARMIŞ ONA GÖRE İŞLEM YAPTIGIM İÇİN CLASSİFİCATİON REPORT - CONFUSİON MATRİX -GRAD CAM HEATMAP oluşturmada sıkıntılar yaşadım o yuzden test yerine validation setini kullanıp işlemlerimi tekrarlayacağım.

Modelin test verisi üzerindeki performansını daha detaylı incelemek için sınıf bazlı metrikler ve karışıklık matrisi hesaplanmıştır. 

Classification Report(projede istenen) test verisinde bulunan sınıf sayısı 1 oldugu için 
F1-score: Precision ve Recall’un dengeli bir birleşimidir. Özellikle dengesiz sınıf dağılımlarında önemlidir.

Support: Her sınıf için test setinde kaç örnek bulunduğunu belirtir.

Confusion Matrix (Karışıklık Matrisi):
Satırlar = Gerçek etiketler
Sütunlar = Modelin tahmin ettiği etiketler

Yorum:
Eğer matriste diyagonal çok baskınsa → model iyi sınıflandırıyor.
Eğer bazı sınıflar birbirine karışıyorsa → veri sayısı artırılabilir, data augmentation uygulanabilir veya model derinliği değiştirilebilir.


!!!! "Test setinde tüm sınıflar bulunmadığı için rapor sadece testte yer alan sınıflar üzerinden hazırlanmıştır."

In [ ]:
test_loss, test_acc = model.evaluate(test_ds)
print("Test Accuracy:", test_acc)
print("Test Loss:", test_loss)

33 değerlikli test setim olduğunu eveluate fonksiyonu ile anladım ve bu yuzden confusion matrix ve classification report yazarken hata aldıgımı anlamıs oldum.

In [ ]:
# 2) Confusion Matrix & Classification Report
# --------------------------
# Validation setten gerçek ve tahmin etiketlerini al
y_true = np.concatenate([y.numpy() for x, y in val_ds], axis=0)
y_pred = np.concatenate([model.predict(x) for x, y in val_ds], axis=0)

y_true = y_true.argmax(axis=1)   # One-hot vektörleri sınıf indeksine çevir
y_pred = y_pred.argmax(axis=1)

# Rapor yazdır
print(classification_report(y_true, y_pred, target_names=val_ds.class_names))

# Confusion matrix çizdir
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(12,10))
sns.heatmap(cm, annot=False, cmap="Blues",
            xticklabels=val_ds.class_names,
            yticklabels=val_ds.class_names)
plt.title("Confusion Matrix (Validation Set)")
plt.show()


In [ ]:
from sklearn.metrics import classification_report

report = classification_report(y_true, y_pred, target_names=val_ds.class_names, output_dict=True)

print("Macro Avg:", report["macro avg"])
print("Weighted Avg:", report["weighted avg"])
print("Accuracy:", report["accuracy"])


Bunun daha duzenli bir halini dataframe e çevirererek sunmak istersem eğer...

In [ ]:
report_df = pd.DataFrame(report).transpose()
report_df = report_df.round(3)  # 3 ondalık basamak
report_df.head(10)  


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(12,10))
sns.heatmap(cm, annot=False, cmap="Blues",
            xticklabels=val_ds.class_names,
            yticklabels=val_ds.class_names)
plt.title("Confusion Matrix (Validation Set)")
plt.show()

y_true) (y_pred) karşılaştırıyor.Kaç örneğin doğru/yanlış sınıflandırıldığını tablo halinde çıkarıyor.

Her hücrede sayı var → kaç örneğin o sınıfa düştüğünü gösteriyor.Mavi tonları: sayı arttıkça hücre daha koyu renge boyanıyor.

xticklabels ve yticklabels
X ekseni: Modelin tahmin ettiği sınıflar
Y ekseni: Gerçek sınıflar
